## ComfyUI起動

In [ ]:
import os
import socket
import time
from dotenv import load_dotenv

def wait_for_port(host, port, timeout=20):
    start_time = time.time()
    while time.time() - start_time < timeout:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.connect((host, port))
                return True
            except (ConnectionRefusedError, OSError):
                time.sleep(0.5)
    return False

# 2. ffxvsのネガティブプロンプトパックをクローン
%cd ~/
!git clone https://huggingface.co/ffxvs/negative-prompts-pack
%cd negative-prompts-pack
!git lfs pull

# 3. ComfyUIのembeddingフォルダへコピー
!mkdir -p ~/ComfyUI/models/embeddings/SD1.5
!mv *.pt *.safetensors ~/ComfyUI/models/embeddings/SD1.5

# 4. cloneしたリポジトリを削除して後始末
%cd ~/
!rm -rf negative-prompts-pack

import subprocess

if CLOUD_GPU == "LightningAI":
    import torch

    torch_version = torch.__version__
    cpu_flag = "--cpu" if "cpu" in torch_version.lower() else None
    jst = dict(os.environ, TZ="JST-9")
    comfy_options = ["python", "ComfyUI/main.py"]
    if cpu_flag:
        comfy_options.append(cpu_flag)

    print(comfy_options)
    subprocess.Popen(comfy_options, env=jst)
    clear_output()
    print("✅ ComfyUI 起動完了")

    if wait_for_port("127.0.0.1", 8188):
        print("ComfyUI が起動してポートをListenしています。")
    else:
        print("ポート8188に接続できません。起動に失敗している可能性があります。")

#    clear_output()

    display(HTML("""
        <h3>🚀 Launch ComfyUI</h3>
        <a href="http://localhost:8188" target="_blank">
        <button style="font-size:16px;padding:10px 16px;">Open ComfyUI</button>
        </a>
    """))

elif CLOUD_GPU == "RunPod":
    subprocess.Popen(["python", "main.py", "--listen", "--port", "3000"], cwd="/ComfyUI")
